In this project, as input we have some dummy data of a e-shop log including customer ids (regular customer ids start with 'CC' prefix, product ids (in the form of categoryId_itemId), and the total duration that each customer spent when clicking on an item. We can use the original duration in seconds (from file customer_product_clickDuration.csv) or the scaled duration (in range 0-1) (from file customer_product_clickDuration_scaled.csv). Moreover, we want to inform the customers for marketing reasons about a product via email, but there is no such information in the input data. So, we will create dummy emails for the sake of this project.

We will use Surprise library which is a python library for machine learning purposes, to run a series of prediction algorithms (SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering) and choose the one with the lowest RMSE. We will then make a list of regular customers that are recommended to be most probably interested in a specific product (e.g so that they can be informed if the e-shop wishes to promote this specific product).

Then we will use Turicreate library in order to run content-based popularity model and collaborative filtering predictive models (cosine similarity and pearson coefficient). For the specific data the cosine similarity model seems to give the best results (lower RMSE and precision-recall closer to 1), so it will be chosen to create a list of the ten most suitable products to recommend to each customer.

## 1. Import modules

In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import csv

import sys
sys.path.append("..")

# for fake emails
import random
import string
import progressbar

import turicreate as tc
from sklearn.model_selection import train_test_split as t_train_test_split
import time

In [2]:
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor
from surprise import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import BaselineOnly, CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split as s_train_test_split

## 2. Import and manipulate data

In [3]:
orig_data = pd.read_csv('../customer_product_clickDuration.csv', header=None, names=['Customer', 'Product', 'Duration']) 
scaled_data = pd.read_csv('../customer_product_clickDuration_scaled.csv', header=None, names=['Customer', 'Product', 'Duration'])

In [4]:
orig_data.head()

,Customer,Product,Duration
0,CC20190310,0_0,146.0
1,CC20190310,0_1,8.0
2,CC20190310,0_10,89.0
3,CC20190310,0_11,33.0
4,CC20190310,0_12,38.0


In [5]:
orig_data.shape

(178122, 3)

In [6]:
# Create fake emails
'''
Creates a random string of digits between 1 and 20 characters alphanumeric and adds it to a fake domain and fake 
extension
Most of these emails are completely bogus (eg - gmail.gov) but will meet formatting requirements
'''
def makeEmail():
    extensions = ['com','net','org','gov']
    domains = ['gmail','yahoo','comcast','verizon','charter','hotmail','outlook','frontier']

    winext = extensions[random.randint(0,len(extensions)-1)]
    windom = domains[random.randint(0,len(domains)-1)]

    acclen = random.randint(1,20)

    winacc = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(acclen))

    finale = winacc + "@" + windom + "." + winext
    return finale

# create custEmails dataframe with regular customers' ids and emails (below)
custEmails = pd.DataFrame(orig_data[orig_data['Customer'].str.startswith('CC')]['Customer'].drop_duplicates())

#save count to var howmany (only regular customers)
howmany = len(custEmails)

counter = 0      #counter for While loop
emailarray = []  #empty array for loop

print "Creating email addresses..."

prebar = progressbar.ProgressBar(maxval=int(howmany))

for i in prebar(range(howmany)):
    while counter < howmany:
        emailarray.append(str(makeEmail()))
        counter = counter+1
        prebar.update(i)
    
print "Email creation completed."

bar = progressbar.ProgressBar(maxval=int(howmany))
custEmails['Email'] = emailarray

# save regular customer Ids and Emails in custEmails.csv
custEmails.to_csv('../custEmails.csv',index = None, header=False)

custEmails.head()

Creating email addresses...
Email creation completed.


100% |########################################################################|


,Customer,Email
0,CC20190310,ss681qwe7s6e8fv@verizon.gov
215,CC201903100,rfomv8oui34kgf038ra@gmail.gov
437,CC201903101,txn5ynoya@gmail.com
589,CC201903102,vl07knaojbny9mdd@comcast.gov
808,CC201903103,l4e4086bdksupaly@outlook.org


In [7]:
custEmails.shape

(564, 2)

## 3. Recommendation based on highest click duration (highest number of seconds of a product visit)

In [29]:
# find the 10 most interested customers in a specific product based on the time they spent on product visit
orig_data[orig_data['Product']=='0_10'].sort_values(by='Duration', ascending=False).head(10)

,Customer,Product,Duration
110849,CC20190576,0_10,387.0
49620,CC2019042,0_10,359.0
17843,CC201903236,0_10,253.0
27734,CC20190337,0_10,250.0
79130,CC201905126,0_10,249.0
79778,CC201905130,0_10,247.0
95439,CC201905238,0_10,244.0
69939,CC20190463,0_10,236.0
73617,CC20190486,0_10,234.0
102018,CC201905275,0_10,232.0


## 4. Predictions with Surprise library

In [8]:
# find maximum Duration value in order to set the range of rating_scale below
max(orig_data['Duration'])

489.0

In [9]:
reader = Reader(rating_scale=(0, 489))

# load data as Dataset for surprise library
data = Dataset.load_from_df(orig_data[['Customer', 'Product', 'Duration']], reader)

In [10]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    print('Executing' + str(algorithm))
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Executing<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f8f234dab90>
Executing<surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7f8f234dac10>
Executing<surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7f8f234dac50>
Executing<surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7f8f234dac90>
Executing<surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7f8f234dacd0>
Executing<surprise.prediction_algorithms.knns.KNNBaseline object at 0x7f8f234dad10>
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Executing<surprise.prediction_algorithms.knns.KNNBasic object at 0x7f8f234dad50>
Computing the msd similarity matrix...
Done computing similarity

,fit_time,test_rmse,test_time
Algorithm,,,
SlopeOne,0.735442,30.816133,9.561536
CoClustering,3.470109,30.968772,0.978340
KNNWithMeans,4.617046,31.036756,53.858015
BaselineOnly,0.494018,31.181225,1.136886
KNNWithZScore,5.160181,31.193441,56.495376
KNNBaseline,5.327807,31.278019,56.979165
KNNBasic,4.637901,34.669724,54.674279
NormalPredictor,0.313433,64.905987,1.185403
NMF,10.141021,64.929049,1.029189


### Start procedure with the best algorithm (according to the above results, choose the algo with the lowest rmse)

#### Run cross-validation with the best algo (in the specific case SlopeOne) 

In [14]:
# SlopeOne algorithm gave us the best rmse, therefore, we will train and predict with SlopeOne 

print('Executing SlopeOne')
algo = SlopeOne()
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Executing SlopeOne


{u'fit_time': (0.6836080551147461, 0.6862411499023438, 0.6941580772399902),
 u'test_rmse': array([30.92143762, 30.81119719, 30.72042653]),
 u'test_time': (9.618637084960938, 9.231590986251831, 9.458172082901001)}

#### Train and test the chosen algorithm

In [15]:
trainset, testset = s_train_test_split(data, test_size=0.25)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 30.7936


30.79359513131922

#### Get detailed results for predictions/recommendations

In [16]:
#  inspect our predictions in details

def get_Iu(uid):
    """ return the number of items clicked by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items clicked by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have clicked given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have clicked the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [27]:
# print the 10 best predictions
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
37875,CC20190399,7_24,95.0,94.999933,{u'was_impossible': False},168,591,0.000067
17150,RC5274082,3_3,20.0,20.000170,{u'was_impossible': False},60,604,0.000170
1518,RC9213881,0_19,13.0,12.998187,{u'was_impossible': False},30,632,0.001813
8675,RC7910948,4_27,19.0,19.002411,{u'was_impossible': False},53,400,0.002411
26243,RC7577385,1_20,21.0,21.005739,{u'was_impossible': False},93,603,0.005739
42218,RC7305871,7_6,19.0,19.006251,{u'was_impossible': False},50,592,0.006251
4246,CC20190541,3_4,55.0,54.993737,{u'was_impossible': False},158,590,0.006263
27244,RC9305698,5_26,21.0,21.006595,{u'was_impossible': False},60,418,0.006595
29383,RC2844097,5_18,20.0,19.993065,{u'was_impossible': False},24,581,0.006935
9023,CC20190314,7_26,61.0,60.992896,{u'was_impossible': False},158,606,0.007104


In [28]:
# print the 10 worst predictions
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
30087,CC20190466,5_24,344.0,144.057297,{u'was_impossible': False},167,594,199.942703
20427,CC201903184,0_6,375.0,172.024297,{u'was_impossible': False},166,594,202.975703
13879,CC20190337,7_23,414.0,206.465620,{u'was_impossible': False},157,594,207.534380
9389,CC201905238,0_25,358.0,150.172520,{u'was_impossible': False},163,610,207.827480
7639,CC20190514,3_23,329.0,116.288613,{u'was_impossible': False},171,581,212.711387
17668,CC201905115,0_11,374.0,161.176710,{u'was_impossible': False},169,579,212.823290
30814,CC20190451,0_13,391.0,177.592189,{u'was_impossible': False},170,598,213.407811
24790,CC201904276,3_25,319.0,96.677639,{u'was_impossible': False},163,630,222.322361
1566,CC201904171,7_9,437.0,214.547553,{u'was_impossible': False},154,620,222.452447
38185,CC201903288,0_16,391.0,152.319574,{u'was_impossible': False},174,619,238.680426


In [ ]:
# test example: regular customers that are recommended to be most probably interested in product 0_10
listOfCustomers = pd.DataFrame(df[(df['iid'] == '0_10') & (df['uid'].str.startswith('CC'))]['uid'])

## 5. Predictions with Turicreate library

#### Define train and test subsets

In [30]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = t_train_test_split(data, test_size = .25)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [31]:
train_data, test_data = split_data(orig_data)

In [32]:
# define the variables to use in the models
# custEmails.csv has the ids and emails of the regular customers

customersEmails = pd.read_csv('../custEmails.csv', header=None, names=['Customer', 'Email']) 
user_id = 'Customer'
item_id = 'Product'
users_to_recommend = list(customersEmails['Customer'])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [34]:
# function for all models using Turicreate

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
recom = model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)
return model

#### Train popularity, cosine and pearson models with trainset

In [35]:
# Content-based popularity model

name = 'popularity'
target = 'Duration'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 133591 observations with 1436 users and 235 items.

Data prepared in: 0.309455s

133591 observations to process; with 235 unique items.

+-------------+---------+---------------+------+
|   Customer  | Product |     score     | rank |
+-------------+---------+---------------+------+
|  CC20190310 |   0_1   |  56.563237774 |  1   |
|  CC20190310 |   2_15  | 56.3128038898 |  2   |
|  CC20190310 |   7_5   | 56.1879194631 |  3   |
|  CC20190310 |   2_25  | 56.1398373984 |  4   |
|  CC20190310 |   1_23  | 56.0455311973 |  5   |
|  CC20190310 |   6_19  | 56.0306122449 |  6   |
|  CC20190310 |   0_13  | 55.7516339869 |  7   |
|  CC20190310 |   4_11  | 55.4221854305 |  8   |
|  CC20190310 |   3_24  | 55.3798319328 |  9   |
|  CC20190310 |   1_21  | 55.3282571912 |  10  |
| CC201903100 |   2_29  | 58.5418994413 |  1   |
| CC201903100 |   4_10  | 57.5108153078 |  2   |
| CC201903100 |   0_25  | 56.8273716952 |  3   |
| CC201903100 |   0_6   |     56.44     |  4   |
| CC201903100 |   0_4   | 56.1888341544 |  5   |
| CC201903100 |   2_25  | 56.1398373984 |  6   |
| CC201903100 |   3_0   | 55.9748743719 |  7   |
| CC201903100 |   1_

In [36]:
# Collaborative Filtering Model - Cosine similarity

name = 'cosine'
target = 'Duration'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 133591 observations with 1436 users and 235 items.

Data prepared in: 0.248174s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------+---------+---------------+------+
|   Customer  | Product |     score     | rank |
+-------------+---------+---------------+------+
|  CC20190310 |   5_28  | 54.7382656228 |  1   |
|  CC20190310 |   2_25  | 51.0554222165 |  2   |
|  CC20190310 |   6_13  | 43.6654091707 |  3   |
|  CC20190310 |   1_16  | 35.9204038622 |  4   |
|  CC20190310 |   1_7   |  22.949102294 |  5   |
|  CC20190310 |   7_19  | 19.6976116551 |  6   |
|  CC20190310 |   7_21  | 17.5764856131 |  7   |
|  CC20190310 |   6_7   | 16.9021260142 |  8   |
|  CC20190310 |   1_23  | 16.8149473815 |  9   |
|  CC20190310 |   7_10  | 16.8113241203 |  10  |
| CC201903100 |   5_1   | 188.822367826 |  1   |
| CC201903100 |   2_25  | 79.5419086751 |  2   |
| CC201903100 |   5_28  |  59.735086386 |  3   |
| CC201903100 |   6_13  | 56.4555488839 |  4   |
| CC201903100 |   3_21  | 40.0138619197 |  5   |
| CC201903100 |   4_21  | 32.1620058647 |  6   |
| CC201903100 |   1_7   | 32.0558597825 |  7   |
| CC201903100 |   3_

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.12203s

In [37]:
# Collaborative Filtering Model - Pearson similarity

name = 'pearson'
target = 'Duration'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 133591 observations with 1436 users and 235 items.

Data prepared in: 0.255093s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------+---------+---------------+------+
|   Customer  | Product |     score     | rank |
+-------------+---------+---------------+------+
|  CC20190310 |   0_1   | 56.8112906239 |  1   |
|  CC20190310 |   2_25  |  56.667460248 |  2   |
|  CC20190310 |   7_5   | 56.4509691707 |  3   |
|  CC20190310 |   2_15  | 56.1829815001 |  4   |
|  CC20190310 |   0_13  | 55.9527335937 |  5   |
|  CC20190310 |   4_11  |  55.939123704 |  6   |
|  CC20190310 |   3_24  | 55.8466923126 |  7   |
|  CC20190310 |   1_23  | 55.5762703888 |  8   |
|  CC20190310 |   6_19  | 55.4778961356 |  9   |
|  CC20190310 |   1_21  | 55.3871078261 |  10  |
| CC201903100 |   2_9   | 65.9497916875 |  1   |
| CC201903100 |   0_25  | 65.7001871626 |  2   |
| CC201903100 |   0_4   | 64.9097636993 |  3   |
| CC201903100 |   1_21  | 63.6955871634 |  4   |
| CC201903100 |   0_15  | 62.6495676655 |  5   |
| CC201903100 |   3_24  | 62.6097614887 |  6   |
| CC201903100 |   4_10  | 61.2921797791 |  7   |
| CC201903100 |   4_

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.407191s

In [38]:
# Model Evaluation

models_w_counts = [popularity, cos, pear]
names_w_counts = ['Popularity Model on Click Duration', 'Cosine Similarity on Click Duration', 'Pearson Similarity on Click Duration']

# compare all the models we have built based on RMSE and precision-recall
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Click Duration


recommendations finished on 1000/1436 queries. users per second: 75901.3


Precision and recall summary statistics by cutoff
+--------+----------------+------------------+
| cutoff | mean_precision |   mean_recall    |
+--------+----------------+------------------+
|   1    | 0.122562674095 | 0.00306971181551 |
|   2    | 0.12708913649  | 0.00647469810327 |
|   3    | 0.211234911792 | 0.0161977820342  |
|   4    | 0.246344011142 | 0.0253531988764  |
|   5    | 0.269777158774 | 0.0346912084171  |
|   6    | 0.283194057567 | 0.0437458093992  |
|   7    | 0.296557898926 | 0.0531406721497  |
|   8    | 0.305013927577 |  0.062336792465  |
|   9    | 0.312674094708 | 0.0716364851583  |
|   10   | 0.319359331476 | 0.0813166328221  |
+--------+----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 48.5127487353

Per User RMSE (best)
+-----------+-------+---------------+
|  Customer | count |      rmse     |
+-----------+-------+---------------+
| RC7000141 |   9   | 17.3042505948 |
+-----------+-------+---------------+
[1 rows x 3 columns]


Per 

recommendations finished on 1000/1436 queries. users per second: 28219.1


Precision and recall summary statistics by cutoff
+--------+----------------+------------------+
| cutoff | mean_precision |   mean_recall    |
+--------+----------------+------------------+
|   1    | 0.327994428969 | 0.00865654812839 |
|   2    | 0.300835654596 | 0.0166682993959  |
|   3    | 0.305246053853 | 0.0256119608563  |
|   4    | 0.312674094708 |  0.034383063043  |
|   5    | 0.323398328691 |  0.043940685616  |
|   6    | 0.33286908078  | 0.0539656451829  |
|   7    | 0.339037007561 | 0.0629904330081  |
|   8    | 0.343662952646 | 0.0720706764197  |
|   9    | 0.348653667595 | 0.0817051533343  |
|   10   | 0.35069637883  | 0.0913664014689  |
+--------+----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 67.3593676706

Per User RMSE (best)
+-----------+-------+---------------+
|  Customer | count |      rmse     |
+-----------+-------+---------------+
| RC6959525 |   3   | 3.37656498624 |
+-----------+-------+---------------+
[1 rows x 3 columns]


Per 

recommendations finished on 1000/1436 queries. users per second: 24431.4


Precision and recall summary statistics by cutoff
+--------+----------------+------------------+
| cutoff | mean_precision |   mean_recall    |
+--------+----------------+------------------+
|   1    | 0.24651810585  | 0.00551948679498 |
|   2    | 0.243036211699 | 0.0108815887417  |
|   3    | 0.262999071495 | 0.0178321064146  |
|   4    | 0.275766016713 | 0.0248559919287  |
|   5    | 0.289972144847 | 0.0331445606643  |
|   6    | 0.300951717734 | 0.0420003938706  |
|   7    | 0.309689614007 | 0.0510143410369  |
|   8    | 0.317896935933 | 0.0607266223435  |
|   9    | 0.322810275457 | 0.0697264369983  |
|   10   | 0.325348189415 | 0.0782961235013  |
+--------+----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 45.662402753

Per User RMSE (best)
+-----------+-------+---------------+
|  Customer | count |      rmse     |
+-----------+-------+---------------+
| RC7000141 |   9   | 16.0501072159 |
+-----------+-------+---------------+
[1 rows x 3 columns]


Per U

In [39]:
# Final Output (cosine model seems to give best results)

final_model = tc.item_similarity_recommender.create(tc.SFrame(orig_data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='Duration', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 178122 observations with 1436 users and 235 items.

Data prepared in: 0.339727s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

+-------------+---------+---------------+------+
|   Customer  | Product |     score     | rank |
+-------------+---------+---------------+------+
|  CC20190310 |   5_28  | 14.4362423179 |  1   |
|  CC20190310 |   1_7   | 13.6939289955 |  2   |
|  CC20190310 |   1_25  |  13.089405558 |  3   |
|  CC20190310 |   1_27  | 11.5935009918 |  4   |
|  CC20190310 |   4_29  | 9.26696092528 |  5   |
|  CC20190310 |   1_21  | 8.84142201584 |  6   |
|  CC20190310 |   6_27  |  8.6192780941 |  7   |
|  CC20190310 |   7_29  | 8.56741336584 |  8   |
|  CC20190310 |   4_5   | 8.45890017609 |  9   |
|  CC20190310 |   7_21  | 7.87894510003 |  10  |
| CC201903100 |   1_25  | 27.1173051024 |  1   |
| CC201903100 |   5_28  | 23.4523337588 |  2   |
| CC201903100 |   1_27  | 19.9371107027 |  3   |
| CC201903100 |   4_29  | 17.5181095791 |  4   |
| CC201903100 |   7_29  |  13.44845051  |  5   |
| CC201903100 |   7_27  | 12.0275822301 |  6   |
| CC201903100 |   1_26  | 9.62542655983 |  7   |
| CC201903100 |   0_

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.168775s